In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [2]:
train_data = pd.read_json('./data/train.json')
test_data = pd.read_json('./data/test.json')

print(train_data.head())
print(train_data.shape)
print(test_data.head())
print(test_data.shape)


                                              band_1  \
0  [-27.878360999999998, -27.15416, -28.668615, -...   
1  [-12.242375, -14.920304999999999, -14.920363, ...   
2  [-24.603676, -24.603714, -24.871029, -23.15277...   
3  [-22.454607, -23.082819, -23.998013, -23.99805...   
4  [-26.006956, -23.164886, -23.164886, -26.89116...   

                                              band_2        id inc_angle  \
0  [-27.154118, -29.537888, -31.0306, -32.190483,...  dfd5f913   43.9239   
1  [-31.506321, -27.984554, -26.645678, -23.76760...  e25388fd   38.1562   
2  [-24.870956, -24.092632, -20.653963, -19.41104...  58b2aaa0   45.2859   
3  [-27.889421, -27.519794, -27.165262, -29.10350...  4cfc3a18   43.8306   
4  [-27.206915, -30.259186, -30.259186, -23.16495...  271f93f4   35.6256   

   is_iceberg  
0           0  
1           0  
2           1  
3           0  
4           0  
(1604, 5)
                                              band_1  \
0  [-15.863251, -15.201077, -17.887735, -19.

In [3]:
# Training data
x1_train = np.array([np.array(band).astype(np.float32).reshape(75, 75, 1) for band in train_data["band_1"]])
x2_train = np.array([np.array(band).astype(np.float32).reshape(75, 75, 1) for band in train_data["band_2"]])

# Testing data
x1_test = np.array([np.array(band).astype(np.float32).reshape(75, 75, 1) for band in test_data["band_1"]])
x2_test = np.array([np.array(band).astype(np.float32).reshape(75, 75, 1) for band in test_data["band_2"]])


In [ ]:
x3_train = np.array([residue_1(x1_train[i]) for i in range(x1_train.shape[0])])
x4_train = np.array([residue_1(x2_train[i]) for i in range(x2_train.shape[0])])
X_train = np.concatenate([x1_train, x2_train, x3_train, x4_train], axis=-1)


In [10]:
X_train = np.concatenate([x1_train, x2_train,(x1_train+x2_train)/2], axis=-1)
Y_train = np.array(train_data["is_iceberg"])
X_test = np.concatenate([x1_test, x2_test], axis=-1)


In [11]:
from keras.preprocessing.image import ImageDataGenerator

normalize1 = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization = True
)
normalize1.fit(X_train)
X_train = next(normalize1.flow(X_train,None,batch_size=X_train.shape[0],shuffle=False))




In [12]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras import regularizers
from keras.layers import Conv2D, SeparableConv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization



# Create model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(75,75,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))



In [ ]:
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

X = X_train
Y = Y_train
for train, test in kfold.split(X, Y):
    model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
    model.fit(X[train], Y[train],epochs=10)
    scores = model.evaluate(X[test], Y[test], verbose=1)
    print('\n',scores)


Epoch 1/10
1282/1282 [==============================] - 32s - loss: 1.2956 - acc: 0.5359    
Epoch 2/10
1282/1282 [==============================] - 30s - loss: 0.6188 - acc: 0.6825    
Epoch 3/10
1282/1282 [==============================] - 31s - loss: 0.4750 - acc: 0.7668    
Epoch 4/10
1282/1282 [==============================] - 31s - loss: 0.3753 - acc: 0.8253    
Epoch 5/10
1282/1282 [==============================] - 31s - loss: 0.3076 - acc: 0.8775    
Epoch 6/10
1282/1282 [==============================] - 31s - loss: 0.1933 - acc: 0.9314    
Epoch 7/10
1282/1282 [==============================] - 31s - loss: 0.0999 - acc: 0.9688    
Epoch 8/10
1282/1282 [==============================] - 32s - loss: 0.0767 - acc: 0.9789    
Epoch 9/10
1282/1282 [==============================] - 32s - loss: 0.0446 - acc: 0.9891    
Epoch 10/10
320/322 [============================>.] - ETA: 0s
 [0.39424761387626917, 0.85403726708074534]
Epoch 1/10
 192/1283 [===>..........................] - 

In [ ]:
print(model.predict(X_train[0:10,:,:,:]))
print(Y_train[0:10])

In [ ]:
prediction = model.predict(X_test, verbose=1)
submit_df = pd.DataFrame({'id': test_data["id"], 'is_iceberg': prediction.flatten()})
submit_df.head()

In [ ]:
submit_df.to_csv("./naive_submission.csv", index=False)